In [ ]:
%run NB - Load Configuration

### **Enter the workspace name the notebooks to be monitored are located and also the name of the notebooks to be monitored**

In [ ]:
# Workspace list
workspace_names = ['OnyxTools-Test','OnyxTools-Dev']

In [ ]:
# Notebook list
monitoring_list = ['NB - Dummy Data', 'NB - Sempy Retrieves IDs']

In [ ]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException
import pandas as pd
import uuid

#Instantiate the client
client = fabric.FabricRestClient()

#### Retrieve Workspace IDs

In [ ]:
# Resolve workspace names to IDs, skipping any not found
workspace_ids = {}
for name in workspace_names:
    try:
        wid = fabric.resolve_workspace_id(name)
        workspace_ids[name] = wid
    except WorkspaceNotFoundException:
        print(f"Workspace '{name}' not found. Skipping...")

# Create the workspaces list with both ID and name
workspaces = [{"workspace_id": wid, "workspace_name": name} for name, wid in workspace_ids.items()]


In [ ]:
# Store disabled schedules
disabled_schedules = []

for workspace in workspaces:
    workspace_id = workspace["workspace_id"]
    workspace_name = workspace["workspace_name"]

    try:
        items_url = f"/v1/workspaces/{workspace_id}/items"
        items_response = client.get(items_url)

        if items_response.status_code != 200:
            continue

        items_data = items_response.json()
        items = items_data.get('value', [])

        workspace_notebooks = [item for item in items if item.get('type') == 'Notebook']

        for notebook in workspace_notebooks:
            notebook_id = notebook.get('id')
            notebook_name = notebook.get('displayName', 'Unnamed Notebook')

            if notebook_name not in monitoring_list:
                continue

            job_type = "RunNotebook"
            schedule_url = f"/v1/workspaces/{workspace_id}/items/{notebook_id}/jobs/{job_type}/schedules"

            try:
                schedule_response = client.get(schedule_url)
                if schedule_response.status_code == 200:
                    schedules = schedule_response.json().get('value', [])

                    for s in schedules:
                        if not s.get("enabled", False):
                            disabled_schedules.append({
                                "WorkspaceID": workspace_id,
                                "WorkspaceName": workspace_name,
                                "NotebookID": notebook_id,
                                "NotebookName": notebook_name,
                                "ScheduleID": s.get("id", "Unknown"),
                                "Status": "Disabled",
                                "DetectedAt": pd.Timestamp.now(),
                                "TriggerID": str(uuid.uuid4())
                            })

            except Exception as e:
                print(f"Error checking schedule for {notebook_name}: {e}")

    except Exception as e:
        print(f"Error processing workspace {workspace_name}: {e}")


In [ ]:
if disabled_schedules:
    print(f"✅{len(disabled_schedules)} disabled schedules found")
else:
    print("ℹ️ No disabled schedules found.")

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import pandas as pd
# Convert to Pandas DataFrame
disabled_df = pd.DataFrame(disabled_schedules)
#display(disabled_df)

# Define schema
schema = StructType([
    StructField("WorkspaceID", StringType(), True),
    StructField("WorkspaceName", StringType(), True),
    StructField("NotebookID", StringType(), True),
    StructField("NotebookName", StringType(), True),
    StructField("ScheduleID", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("DetectedAt", TimestampType(), True),
    StructField("TriggerID", StringType(), True)
])

# Convert to Spark DataFrame, even if empty
disabled_spark_df = spark.createDataFrame(disabled_df if not disabled_df.empty else [], schema)
# display(disabled_spark_df)

In [ ]:
import json

output = disabled_spark_df.toJSON().collect()
mssparkutils.notebook.exit(json.dumps([json.loads(row) for row in output]))